### Objectif  

Le but principal de ce fichier est d'organiser les données audio en divisant les enregistrements disponibles en ensembles d'entraînement (**train**) et de **test** pour chaque genre (homme et femme) au sein de chaque langue. Une fois cette séparation réalisée, les **coefficients MFCC** sont extraits pour chaque fichier audio des ensembles respectifs, puis sauvegardés dans des répertoires spécifiques :  

- **train** : contient les sous-dossiers avec les enregistrements destinés à l'entraînement des modèles pour chaque genre et langue.  
- **test** : regroupe les dossiers contenant les données de test des genres de chaque langue.  

Cette approche permet une gestion claire et structurée des données, facilitant ainsi l'entraînement et la validation des modèles de reconnaissance automatique de la langue.

### Importation des bibliothèques

In [3]:
# !pip install python_speech_features

In [5]:
import os
import numpy as np
import pandas as pd
import re # Pour découper les noms de fichiers audio
import shutil # Pour déplacer des fichiers
import scipy.io.wavfile as wav # Pour lire les fichiers audio WAV
from python_speech_features import mfcc # Pour extraire les coefficients MFCC
import wave # Pour vérifier si un fichier est un fichier WAV
import soundfile as sf # Pour lire les fichiers audio WAV


from google.colab import drive
drive.mount('/content/drive') # Monter (Mount) Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Le répertoire de données qui contient les données originales avant la division

In [6]:
# Le répertoire de données repertoireDataSet
repertoireDataSet = '/content/drive/My Drive/Reconnaissance/DataSet/'

langues = os.listdir(repertoireDataSet)
print(langues)

['Spanish', 'French', 'arabic', 'Japanese', 'English']


Le nombre total de fichiers audio pour chaque langue avant la division

In [ ]:
# Compte et affiche le nombre de fichiers audio pour chaque langue et genre :

def compter_fichiers_audio(repertoire_principal):
  for langue in os.listdir(repertoire_principal):
    langue_path = os.path.join(repertoire_principal, langue)

    # Initialisation des compteurs
    nb_fichiers_homme = 0
    nb_fichiers_femme = 0

    for genre in os.listdir(langue_path):
      genre_path = os.path.join(langue_path, genre)

      # Compte les fichiers audio dans le dossier du genre
      nb_fichiers_genre = len([f for f in os.listdir(genre_path) if os.path.isfile(os.path.join(genre_path, f)) and f.lower().endswith(('.wav', '.mp3'))])  # Adaptez les extensions si besoin

      if genre.lower() == "homme":
        nb_fichiers_homme = nb_fichiers_genre
      elif genre.lower() == "femme":
        nb_fichiers_femme = nb_fichiers_genre

    # Affiche les résultats pour la langue
    total_langue = nb_fichiers_homme + nb_fichiers_femme
    print(f"Langue: {langue}")
    print(f"  - Nombre de fichiers Homme: {nb_fichiers_homme}")
    print(f"  - Nombre de fichiers Femme: {nb_fichiers_femme}")
    print(f"  - Total fichiers pour {langue}: {total_langue}")

# Appel de la fonction avec le répertoire principal (repertoireDataSet)
compter_fichiers_audio(repertoireDataSet)

Langue: French
  - Nombre de fichiers Homme: 9
  - Nombre de fichiers Femme: 17
  - Total fichiers pour French: 26
Langue: arabic
  - Nombre de fichiers Homme: 262
  - Nombre de fichiers Femme: 555
  - Total fichiers pour arabic: 817
Langue: Japanese
  - Nombre de fichiers Homme: 999
  - Nombre de fichiers Femme: 676
  - Total fichiers pour Japanese: 1675
Langue: English
  - Nombre de fichiers Homme: 1251
  - Nombre de fichiers Femme: 1452
  - Total fichiers pour English: 2703
Langue: Spanish
  - Nombre de fichiers Homme: 359
  - Nombre de fichiers Femme: 365
  - Total fichiers pour Spanish: 724


### Division des enregistrements de chaque genre pour chaque langue en ensembles train et test

Cette fonction analyse les noms de fichiers audio pour extraire des informations sur le locuteur, le genre et le segment d'enregistrement. Elle renvoie un dictionnaire associant chaque locuteur à ses segments.
- Voici un exemple de sortie de cette fonction pour le genre "**Femme**" de la langue "**Arabic**" : **Ar_F1_1.wav**.

In [7]:
def extraire_locuteur_et_segments(fichiers_audio):
    # Dictionnaire pour stocker les locuteurs et leurs segments associés
    locuteurs_segments = {}

    # Parcours de chaque fichier audio pour extraire les locuteurs et segments
    for fichier in fichiers_audio:
        # Séparation du nom du fichier en utilisant les caractères '_' et ' '
        segments = re.split(r'[_ ]', fichier)
        segments[1] = segments[1].upper()  # Conversion du locuteur en majuscules

        # Cas où le nom est séparé en 3 segments (locuteur, genre, et autre)
        if len(segments) == 3:
            # Filtrer les locuteurs ayant plus d'un caractère
            if len(segments[1]) > 1:
                # Ajouter le locuteur dans le dictionnaire avec le segment
                if locuteurs_segments.get(segments[1]) is None:
                    locuteurs_segments[segments[1]] = [fichier]
                else:
                    locuteurs_segments[segments[1]].append(fichier)

        # Cas où le nom est séparé en 4 segments (locuteur, genre, index, et autre)
        elif len(segments) == 4:
            # Gestion des locuteurs ayant un identifiant court (1 caractère)
            if len(segments[1]) == 1:
                index_locuteur = segments[1] + segments[2]  # Création d'un index combiné

                # Ajouter le locuteur avec l'index dans le dictionnaire
                if locuteurs_segments.get(index_locuteur) is None:
                    locuteurs_segments[index_locuteur] = [fichier]
                else:
                    locuteurs_segments[index_locuteur].append(fichier)
            else:
                # Ajouter les locuteurs avec plus d'un caractère dans le dictionnaire
                if locuteurs_segments.get(segments[1]) is None:
                    locuteurs_segments[segments[1]] = [fichier]
                else:
                    locuteurs_segments[segments[1]].append(fichier)

    # Retourner le dictionnaire des locuteurs et leurs segments associés
    return locuteurs_segments


- Cette fonction divise **chaque genre de chaque langue** en dossiers "train" et "test" en prenant approximativement 2/3 des enregistrements pour l'entraînement ("train") et 1/3 pour le test ("test").
- Elle supprime les locuteurs avec un seul enregistrement.

In [8]:
def separer_entrainement_test(dossierDataSet):

    # Récupérer la liste des langues disponibles dans le jeu de données.
    liste_langues = os.listdir(dossierDataSet)

    for langue in liste_langues:  # Parcours de chaque langue.

        # Récupérer la liste des genres (homme et femme) pour une langue.
        liste_genres = os.listdir(os.path.join(dossierDataSet, langue))

        for genre in liste_genres:  # Parcours de chaque genre pour une langue.

            # Chemin vers le répertoire contenant les enregistrements du genre.
            chemin_genre = os.path.join(dossierDataSet, langue, genre)

            # Récupérer la liste des enregistrements disponibles pour ce genre.
            enregistrements_genre = os.listdir(chemin_genre)

            # Créer un mapping entre chaque locuteur et ses enregistrements pour ce genre.
            locuteur_enregistrements = extraire_locuteur_et_segments(enregistrements_genre)

            # Création des répertoires pour l'entraînement et les tests.
            dossier_train = os.path.join(chemin_genre, "entrainement")
            dossier_test = os.path.join(chemin_genre, "test")
            os.makedirs(dossier_train, exist_ok=True)
            os.makedirs(dossier_test, exist_ok=True)

            # Préparation pour traiter chaque locuteur du genre.
            chemin_prefixe = chemin_genre + "/"

            for locuteur, enregistrements in locuteur_enregistrements.items():  # Parcours des locuteurs et de leurs segments.

                # Nombre total d'enregistrements pour un locuteur.
                total_enregistrements = len(enregistrements)

                # Ignorer les locuteurs ayant moins de 2 enregistrements.
                if total_enregistrements < 2:
                    print(f"Langue: {langue}, Genre: {genre}, Locuteur: {locuteur} supprimé (moins de 2 enregistrements : {total_enregistrements})")
                    # Suppression des enregistrements de ce locuteur
                    for enregistrement in enregistrements:
                        chemin_fichier = os.path.join(chemin_genre, enregistrement)
                        if os.path.exists(chemin_fichier):
                            os.remove(chemin_fichier)
                    continue

                # Calcul du nombre d'enregistrements pour l'ensemble d'entraînement (2/3 des données).
                nombre_entrainement = int(np.round((2 / 3) * total_enregistrements))

                # Ajout du préfixe au chemin des enregistrements.
                enregistrements = [(chemin_prefixe + enregistrement) for enregistrement in enregistrements]

                # Tri des enregistrements par taille (les plus grands en premier).
                enregistrements = sorted(enregistrements, key=os.path.getsize, reverse=True)

                # Suppression du préfixe pour conserver les noms d'origine.
                enregistrements = [enregistrement.replace(chemin_prefixe, '') for enregistrement in enregistrements]

                # Déplacement des enregistrements vers le dossier d'entraînement.
                for i in range(nombre_entrainement):
                    source = os.path.join(chemin_genre, enregistrements[i])
                    destination = os.path.join(dossier_train, enregistrements[i])
                    shutil.move(source, destination)

                # Déplacement des enregistrements restants vers le dossier de test.
                for i in range(nombre_entrainement, total_enregistrements):
                    source = os.path.join(chemin_genre, enregistrements[i])
                    destination = os.path.join(dossier_test, enregistrements[i])
                    shutil.move(source, destination)


Dévision des enregistrements :
- Après l'exécution de cette ligne, il va créer deux dossiers, train et test, pour chaque genre (homme/femme) de chaque langue.
- Environ 2/3 des enregistrements de chaque locuteur seront placés dans le dossier train et environ 1/3 dans le dossier test.

In [ ]:
### N'exécutez pas cette cellule, car elle a déjà été exécutée et les résultats sont déjà enregistrés et stockés dans Google Drive. Cela prend beaucoup de temps.

# Dévision des enregistrements :
separer_entrainement_test(repertoireDataSet)

Locuteur F2 supprimé (moins de 2 enregistrements : 1)
Locuteur H4 supprimé (moins de 2 enregistrements : 1)
Locuteur H5 supprimé (moins de 2 enregistrements : 1)
Locuteur H6 supprimé (moins de 2 enregistrements : 1)
Dataset divisé en ensembles d'entraînement et de test (locuteurs avec un seul enregistrement supprimés).


Le nombre total de fichiers audio pour chaque langue après la division

In [9]:
# Compte et affiche le nombre de fichiers audio pour l'entraînement et le test, pour chaque genre au sein de chaque langue.

def compter_fichiers_train_test_par_genre(repertoire_principal):

    for langue in sorted(os.listdir(repertoire_principal)):
        langue_dir = os.path.join(repertoire_principal, langue)
        print(f"Langue: {langue}")

        for genre in os.listdir(langue_dir):
            genre_dir = os.path.join(langue_dir, genre)
            print(f"  - Genre : {genre}")

            nb_fichiers_train = 0
            nb_fichiers_test = 0

            for phase in ["train", "test"]:
                phase_dir = os.path.join(genre_dir, phase)
                if os.path.exists(phase_dir):  # Vérifier si le dossier existe
                    nb_fichiers_phase = len([f for f in os.listdir(phase_dir) if os.path.isfile(os.path.join(phase_dir, f)) and f.lower().endswith(('.wav', '.mp3'))])
                    if phase == "train":
                        nb_fichiers_train = nb_fichiers_phase
                    else:
                        nb_fichiers_test = nb_fichiers_phase

            print(f"       - Nombre de fichiers Train: {nb_fichiers_train}")
            print(f"       - Nombre de fichiers Test: {nb_fichiers_test}")
        print("-" * 40)

# repertoireDataSet est le chemin vers votre répertoire principal
compter_fichiers_train_test_par_genre(repertoireDataSet)

Langue: English
  - Genre : Femme
       - Nombre de fichiers Train: 968
       - Nombre de fichiers Test: 484
  - Genre : Homme
       - Nombre de fichiers Train: 833
       - Nombre de fichiers Test: 418
----------------------------------------
Langue: French
  - Genre : Femme
       - Nombre de fichiers Train: 10
       - Nombre de fichiers Test: 6
  - Genre : Homme
       - Nombre de fichiers Train: 3
       - Nombre de fichiers Test: 3
----------------------------------------
Langue: Japanese
  - Genre : Femme
       - Nombre de fichiers Train: 451
       - Nombre de fichiers Test: 225
  - Genre : Homme
       - Nombre de fichiers Train: 665
       - Nombre de fichiers Test: 334
----------------------------------------
Langue: Spanish
  - Genre : homme
       - Nombre de fichiers Train: 240
       - Nombre de fichiers Test: 119
  - Genre : femme
       - Nombre de fichiers Train: 243
       - Nombre de fichiers Test: 122
----------------------------------------
Langue: arabic
  - 

### Extraction des caractéristiques audio à l'aide des coefficients MFCC

La fonction **verifier_format_wav** a pour rôle de vérifier si un fichier donné est un fichier audio valide au **format WAV** (Waveform Audio File Format), si ce n'est pas déjà le cas

In [10]:
def verifier_format_wav(chemin_fichier):

    try:
        # Tentative d'ouverture du fichier WAV pour vérifier sa validité
        with wave.open(chemin_fichier, 'rb') as fichier_wav:
            fichier_wav.getparams()  # Test de lecture des paramètres du fichier WAV
        return True  # Si aucun erreur n'est levée, le fichier est valide
    except (wave.Error, FileNotFoundError):
        # Si une erreur est rencontrée, c'est que le fichier n'est pas valide
        return False

La fonction **convertir_en_wav** a pour rôle de convertir un fichier audio en un fichier au format WAV, en utilisant la bibliothèque **soundfile**

In [11]:
def convertir_en_wav(chemin_entree):
    try:
        # Lire le fichier audio à partir du chemin d'entrée
        donnees_audio, frequence_echantillonnage = sf.read(chemin_entree)

        # Sauvegarder le fichier en format WAV en utilisant les mêmes données et la fréquence d'échantillonnage
        sf.write(chemin_entree, donnees_audio, frequence_echantillonnage)
        print(f"Le fichier a été converti : {chemin_entree}")

    except Exception as erreur:
        # En cas d'erreur lors de la conversion, afficher l'erreur
        print(f"Erreur durant la conversion : {erreur}")

- La fonction **mfcc_features** a pour rôle d’extraire les caractéristiques acoustiques **MFCC** (Mel Frequency Cepstral Coefficients) à partir les fichiers audio.
- Ces caractéristiques sont ensuite sauvegardées sous forme de **fichiers .mfcc**

In [12]:
def extraire_features_mfcc(repertoireDataSet, dossier_mfcc):

    # Liste des langues dans le répertoire de données
    langues = os.listdir(repertoireDataSet)

    # Création du répertoire pour sauvegarder les fichiers MFCC
    os.mkdir(dossier_mfcc)

    for langue in langues:  # Parcourt chaque langue dans la liste

        # Création du répertoire pour la langue dans le répertoire MFCC
        mfcc_langue_dir = os.path.join(dossier_mfcc, langue)
        os.mkdir(mfcc_langue_dir)

        # Liste des genres (homme ou femme) pour chaque langue
        genres = os.listdir(os.path.join(repertoireDataSet, langue))

        # Création des répertoires train et test pour chaque langue
        mfcc_train_dir = os.path.join(mfcc_langue_dir, "train")
        os.mkdir(mfcc_train_dir)

        mfcc_test_dir = os.path.join(mfcc_langue_dir, "test")
        os.mkdir(mfcc_test_dir)

        # Parcours des genres (homme ou femme)
        for genre in genres:  # Parcourt chaque genre d'une langue
            genre_dir = os.path.join(repertoireDataSet, langue, genre)

            # Traitement du répertoire train pour extraire et sauvegarder les MFCC
            train_dir = os.path.join(genre_dir, "train")
            for seg in os.listdir(train_dir):  # Parcours des enregistrements dans le répertoire train
                file = os.path.join(train_dir, seg)

                if not verifier_format_wav(file):
                    convertir_en_wav(file)

                rate, signal = wav.read(file)  # Lecture du fichier audio
                # signal, rate = librosa.load(file, sr=None)

                # Extraction des caractéristiques MFCC et calcul de l'énergie
                mfcc_features = mfcc(signal, samplerate=rate, numcep=13, winlen=0.025, winstep=0.01)
                # mfcc_features = mfcc(signal, samplerate=rate, numcep=13, winlen=0.025, winstep=0.01, nfft=2048)
                energie = np.log(np.sum(np.square(mfcc_features), axis=1))  # Calcul de l'énergie

                # Combine les MFCC et l'énergie en une seule matrice
                mfcc_features_with_energy = np.hstack((mfcc_features, energie.reshape(-1, 1)))

                # Sauvegarde des caractéristiques dans un fichier texte avec l'extension .mfcc
                output_file = mfcc_train_dir + "/" + seg.split('.')[0] + '.mfcc'
                np.savetxt(output_file, mfcc_features_with_energy, delimiter=',')

            # Traitement du répertoire test pour extraire et sauvegarder les MFCC
            test_dir = os.path.join(genre_dir, "test")
            for seg in os.listdir(test_dir):  # Parcours des enregistrements dans le répertoire test
                file = os.path.join(test_dir, seg)

                if not verifier_format_wav(file):
                    convertir_en_wav(file)

                rate, signal = wav.read(file)  # Lecture du fichier audio
                # signal, rate = librosa.load(file, sr=None)

                # Extraction des caractéristiques MFCC et calcul de l'énergie
                mfcc_features = mfcc(signal, samplerate=rate, numcep=13, winlen=0.025, winstep=0.01)
                # mfcc_features = mfcc(signal, samplerate=rate, numcep=13, winlen=0.025, winstep=0.01, nfft=2048)
                energie = np.log(np.sum(np.square(mfcc_features), axis=1))  # Calcul de l'énergie

                # Combine les MFCC et l'énergie en une seule matrice
                mfcc_features_with_energy = np.hstack((mfcc_features, energie.reshape(-1, 1)))

                # Sauvegarde des caractéristiques dans un fichier texte avec l'extension .mfcc
                output_file = mfcc_test_dir + "/" + seg.split('.')[0] + '.mfcc'
                np.savetxt(output_file, mfcc_features_with_energy, delimiter=',')


Extraction des caractéristiques

-	Après l'exécution, un dossier "**Features_MFCC**" est automatiquement créé. Ce dossier contient, pour chaque **langue**, un sous-dossier **train** (qui combine les fichiers des genres Homme et Femme pour l'entraînement) et **test** (qui combine les fichiers des genres Homme et Femme pour les tests). Chaque sous-dossier contient les caractéristiques MFCC sauvegardées sous forme de **fichiers .mfcc**.

In [ ]:
### N'exécutez pas cette cellule, car elle a déjà été exécutée et les résultats sont déjà enregistrés et stockés dans Google Drive. Cela prend beaucoup de temps.

dossier_mfcc = "Features_MFCC"
extraire_features_mfcc(repertoireDataSet, dossier_mfcc)

<ipython-input-124-e2a20308d3c1>:37: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate, signal = wav.read(file)  # Lecture du fichier audio
<ipython-input-124-e2a20308d3c1>:58: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate, signal = wav.read(file)  # Lecture du fichier audio


Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_15_37.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_16_5.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_15_78.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_17_17.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_17_25.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_16_52.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_15_57.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_16_76.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_15_48.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_15_58.wav
Fichier converti : /content/drive/My Drive/DataSet/English/Femme/train/engA_F_17_26.wav
Fichier converti : /content/drive

In [ ]:
# Copier le dossier "Features_MFCC" dans Google Drive

source_path = "Features_MFCC"
destination_path = "/content/drive/My Drive/Reconnaissance/Features_MFCC"

# Copier le dossier et son contenu
shutil.copytree(source_path, destination_path)

In [13]:
# Chemin vers le dossier MFCC dans Google Drive
mfcc_dir = '/content/drive/My Drive/Reconnaissance/Features_MFCC'

In [14]:
# Fonction pour compter le nombre de fichiers .mfcc d'entraînement et de test pour chaque langue après l'extraction des coefficients MFCC

def compter_fichiers_train_test(repertoire):
    nb_train = len([f for f in os.listdir(os.path.join(repertoire, 'train')) if f.endswith('.mfcc')])
    nb_test = len([f for f in os.listdir(os.path.join(repertoire, 'test')) if f.endswith('.mfcc')])
    return nb_train, nb_test

# Parcourir les langues et afficher les résultats
for langue in os.listdir(mfcc_dir):
    langue_dir = os.path.join(mfcc_dir, langue)
    nb_train, nb_test = compter_fichiers_train_test(langue_dir)
    print(f"Langue: {langue}")
    print(f"  - Nombre de fichiers d'entraînement: {nb_train}")
    print(f"  - Nombre de fichiers de test: {nb_test}")
    print("-" * 30)  # Séparateur entre les langues

Langue: French
  - Nombre de fichiers d'entraînement: 13
  - Nombre de fichiers de test: 9
------------------------------
Langue: arabic
  - Nombre de fichiers d'entraînement: 544
  - Nombre de fichiers de test: 273
------------------------------
Langue: Japanese
  - Nombre de fichiers d'entraînement: 1116
  - Nombre de fichiers de test: 559
------------------------------
Langue: Spanish
  - Nombre de fichiers d'entraînement: 483
  - Nombre de fichiers de test: 241
------------------------------
Langue: English
  - Nombre de fichiers d'entraînement: 1801
  - Nombre de fichiers de test: 902
------------------------------


In [15]:
# Pour afficher les 5 premières lignes du fichier MFCC

example_mfcc_file = '/content/drive/My Drive/Reconnaissance/Features_MFCC/arabic/train/Ar_H6_47.mfcc'

# Charger le fichier MFCC
mfcc_data = np.loadtxt(example_mfcc_file, delimiter=',')

print("Exemple de données MFCC (5 premières lignes) :")
mfcc_data[:5]

Exemple de données MFCC (5 premières lignes) :


array([[ 13.96639883, -17.71795405,  -1.86812415,   1.89595288,
          6.17643363,   5.30469671,  -9.90279515,  -9.35475516,
         -5.38354897,   0.37825173, -19.93088721, -10.12818876,
         -4.19754409,   7.18121254],
       [ 14.06995216, -18.40062085,  -4.23946508,  -0.30233822,
         -0.18464138,  -3.19857098, -11.41947045, -10.21773955,
         -4.91696206,  -0.55046889, -14.66663144, -13.32885865,
         -7.92555287,   7.15441596],
       [ 14.06628858, -17.62746553,  -3.08532615,  -0.27400438,
         -0.80302208,  -4.23678295, -10.15269705,  -4.98479515,
         -5.20749624,  -1.13389033,  -6.36836441, -13.24375367,
        -10.44600838,   6.92577086],
       [ 13.89955465, -16.17077693,  -3.29592215,  -3.73160736,
          2.18481293,  -4.4150822 ,  -6.07103321,  -0.32681534,
         -7.94161968,  -5.28200966,  -7.47607662, -13.8112792 ,
        -12.57248563,   6.94349856],
       [ 13.92431961, -16.30303636,  -7.40985481,  -8.85916647,
         -1.39891939

- Le genre est ignoré pour les MFCC car on cherche à créer des modèles de reconnaissance vocale indépendants du locuteur.
- En combinant les données des hommes et des femmes, on crée des modèles plus robustes et généralisables.
- Dans notre cas, le genre n'est pas pertinent, car **l'objectif est la reconnaissance de langue**. Peu importe le locuteur (homme ou femme), l'objectif reste de reconnaître correctement la langue parlée.
- Cependant, si l'objectif est d'identifier le locuteur, le genre devient alors une information importante et doit être pris en compte.